In [205]:
import numpy as np
import pandas as pd
from sklearn.base import BaseEstimator, ClusterMixin

In [391]:
def standardize(labels):
#Standardize numerical feature
    return (labels-labels.mean())/labels.std()

def mode2(a, axis=0, val=False):
#Return mode or mode freq of an array
#code based on scipy.stats.mode
    scores = np.unique(np.ravel(a))      
    testshape = list(a.shape)
    testshape[axis] = 1
    oldmostfreq = np.zeros(testshape)
    oldcounts = np.zeros(testshape)

    for score in scores:
        template = (a == score)
        counts = np.expand_dims(np.sum(template, axis),axis)
        mostfrequent = np.where(counts > oldcounts, score, oldmostfreq)
        oldcounts = np.maximum(counts, oldcounts)
        oldmostfreq = mostfrequent

    return oldcounts[0] if val else mostfrequent[0]


In [282]:
a = pd.read_csv('data/avir-tzek.tsv', sep='\t')
a = a.round(1)
a2 = a.dropna().copy()

for col in a2:
    typ = str(a2[col].dtype)
    if typ in ['int', 'int32', 'int64','float', 'float32', 'float64']:
        a2[col] = standardize(a2[col])

In [48]:
c = pd.read_csv('data/h3zm-ta5h.tsv', sep='\t')
c = c.round(1)
c2 = c.dropna().copy()

for col in c2:
    typ = str(c2[col].dtype)
    if typ in ['int', 'int32', 'int64','float', 'float32', 'float64']:
        c2[col] = standardize(c2[col])

In [403]:
class Feature:
#Feature subclass for better use with KModes
    def __init__(self, featType, labels):
        #values in array form and feature dtype
        self.vals, self.type = labels, featType
        #unique values, their orig indices and counts
        vals, inds, counts = np.unique(self.vals, return_inverse=True, return_counts=True)
        #Feature dictionary for use in itialization
        self.valDict = {v:c for v,c in  zip(vals, counts)}
        #frequency of values in orig indices
        self.valFreq = counts[inds]
        return
    
    def getCenter(self, cntrMask):
    #Get center of feature in given centroid
        #Mean for num features
        if self.type==float:
            return self.vals[cntrMask].mean()
        #Mode for cat features
        else:
            return mode2(self.vals[cntrMask], val=False)        
    
    def getSimilarity(self, cntr, cntrdMask=None):
    #Given centroid center, get dissimilarity
        #Numerical feature use euclidean distance
        if self.type==float: 
            return (self.vals - cntr)**2
        else:
            #Get Weight of points, from WHICH paper?
            weights = (self.valFreq + self.valDict[cntr])/(self.valFreq * self.valDict[cntr])
        #2007 dissimilarity
        if cntrdMask is not None:
            return weights * (1-(self.vals==cntr).astype(int) *((self.vals[cntrdMask] == cntr).sum()/cntrdMask.sum()))
        #Classic dissimilarity
        else:
            return weights * (self.vals!=cntr).astype(int)  
            

#######################################################################################################################

def initCentroids(features, sizes):
#Init centroids based on Cao Paper
    #Empty list of centroids
    cntrds = [0 for i in range(sizes['clstrs'])]
    #Density of each point
    dens = np.zeros(sizes['pts'])
    #Iterate through features
    for feat in features:
        #Update point density
        dens += (feat.valFreq/(sizes['pts']*sizes['clstrs']))
    cntrds[0] = [feat.vals[dens[0].argmax()] for feat in features]
    #Iterate though remaining centroids, adapted from github implementation 
    for ki in range(1, sizes['clstrs']):
        dens2 = np.zeros((ki, sizes['pts']))
        #Iterate through already created centroids
        for kii in range(ki):
            #Similarity btwn cntrds and pts
            sims = np.zeros((sizes['pts'], sizes['ftrs']))
            #Iterate through features
            for f, feat in enumerate(features):
                sims[:,f] = feat.getSimilarity(cntrds[kii][f])
            #Densities * similarites
            dens2[kii] = dens * sims.sum(axis=1)
        #New centroid has max dens*sims from previous centroids
        cntrds[ki] = [feat.vals[np.argmax(np.min(dens2, axis=0))] for feat in features] 
    #Assign intial membship
    cntrds, membship, cost = assignMembship(cntrds, features, sizes)
    return cntrds, membship, cost

def assignMembship(cntrds, features, sizes, membship=None, pred=False):
#Assign membship to centroids, membship=True if dissim is 2007, pred=True for KModes predict
    sims = np.zeros((sizes['pts'], sizes['clstrs']))
    #Iterate through centroids
    for i, cntrd in enumerate(cntrds):
        #Get num and cat feat sims  
        num_feats = np.zeros((sizes['pts'], sizes['n_ftrs']))
        cat_feats = np.zeros((sizes['pts'], sizes['c_ftrs']))
        n, c = 0, 0
        #Iterate thorugh features
        for f, feat in enumerate(features):
            #Num features
            if isinstance(feat.type, (float, int)):
                num_feats[:,n] += feat.getSimilarity(cntrd[f])
                n += 1
            #Cat geatures
            else:
                #Choose which dissim to use
                if membship is not None:
                    #2007 dissim
                    cntrdMask = (membship == i)
                    cat_feats[:,c] += feat.getSimilarity(cntrd[f], cntrdMask)
                else:
                    #Orig disim
                    cat_feats[:,c] += feat.getSimilarity(cntrd[f])
                c += 1
        #Sims of given centroid
        sims[:,i] = num_feats.sum(axis=1) + sizes['gamma'] * cat_feats.sum(axis=1)
    #Choose smallest dissim for each pt
    membship = sims.argmin(axis=1)
    #Cost is sum of smallest dissim
    cost = sims.min(axis=1).sum()
    #If centroid has no assigned pts
    if (np.unique(membship).shape[0] != sizes['clstrs'])&(pred):
        cntrds = newCentroids(cntrds, features, membship, sizes)
        #Recursively assign membship
        assignMemship(cntrds, features, sizes, membship)
    return cntrds, membship, cost

def newCentroids(cntrds, features, membship, sizes):
#Create new centroids if centroid is not present in membship
    #Missing centroids
    missing = np.setdiff1d(range(0,sizes['clstrs']), membship)
    #Largest centroid
    max_cntrd = mode2(membship, val=False)
    #Choose pts from largest centroid
    new_cntrds = np.random.choice(np.where(membship == max_cntrd)[0], missing.shape[0])
    #Replace old centroids with new points
    for m, nc in zip(missing, new_cntrds):
        cntrds[m] = [feat.vals[new_cntrds[nc]] for feat in features]
    return cntrds

def setCentroids(features, membship, sizes, dissim):
#Set centroids from center of each feature
    cntrds = [0 for c in range(sizes['clstrs'])]
    #Iterate through centroids, get approp center for each feature
    for i in range(sizes['clstrs']):
        cntrds[i] = [feat.getCenter(membship==i) for feat in features]
    #2007 dissim, pass membship
    if dissim:
        cntrds, membship, cost = assignMembship(cntrds, features, sizes, membship)
    #Classsic dissim
    else:
        cntrds, membship, cost = assignMembship(cntrds, features, sizes)
    return cntrds, membship, cost
 
def setMembshipToLabels(membship, sizes):
#Convert membship to 2D array
    labels = np.zeros((sizes['pts'], sizes['clstrs']))
    for r, cls, in enumerate(membship):
        labels[r, cls] += 1
    return labels


#######################################################################################################################

class KModes(BaseEstimator, ClusterMixin):
#Implementation of KModes, with help from github implementation
    def __init__(self, n_clusters=5, max_iter=100, gamma=1, dissim=False):
        #Keep all size information in dictionary
        self.sizes = {}
        self.sizes['clstrs'], self.sizes['max_itr'] = n_clusters, max_iter
        self.sizes['gamma'] = 1
        #2007 dissim or classic dissim
        self.dissim = dissim
        return     
    
    def fit(self, X):
    #Fit KModes to data
        #Convert Pandas to np array
        if 'pandas' in str(X.__class__):
            X = X.values
        
        self.sizes['pts'], self.sizes['ftrs'] = X.shape
        #Too few clusters
        assert self.sizes['clstrs'] <= self.sizes['pts']
        #Features to better form
        self.features = np.array([Feature(col.dtype, col) for col in X.T])
        #Save n_ftrs and c_ftrs
        self.sizes['n_ftrs'] = int(np.sum([1 for feat in self.features if isinstance(feat.type, (float, int))]))
        self.sizes['c_ftrs'] = int(self.sizes['ftrs'] - self.sizes['n_ftrs'])
        
        #Init centroids, membship and cost
        cntrds, membship, cost = initCentroids(self.features, self.sizes)
        
        #Iterate until convergence or max iteration reached
        itr, converged = 0, False
        while itr <= self.sizes['max_itr'] and not converged:
            itr += 1
            #Set centroids, membship
            cntrds, new_membship, new_cost = setCentroids(self.features, membship, self.sizes, self.dissim)
            #Check if any pts changed membship
            chngs = np.setdiff1d(new_membship, membship)
            #Convergence conditions
            converged = (chngs.shape[0] == 0) or (ncost >= cost)
            membship, cost = new_membship, new_cost
        
        #Save fit info
        self.membship_ = membship
        self.labels_ = setMembshipToLabels(membship, self.sizes)
        self.centroids_, self.cost_, self.itr_ = cntrds, cost, itr
        return self   
    
    def predict(self, X):
    #Given new data, assignMembship to centroids
        assert hasattr(self, 'centroids_'), "Model not yet fit"
        if 'pandas' in str(X.__class__):
            X = X.values
        new_features = np.array([Feature(col.dtype, col) for col in X.T])
        new_sizes = sizes.copy()
        new_sizes[pts] = X.shape[0]
        cntrds, membship, cost = assignMembship(self.centroids_, new_features, new_sizes, pred=False)
        new_labels = setMembshipToLabels(membship, sizes)
        return(new_labels, cost)
    
    def fit_predict(self, X):
        return self.fit(X).predict(X)


In [404]:
soybean = pd.read_csv('data/soybean.csv', header=None)
x = soybean.drop(35, axis=1)
x = x.astype(object)
soyClusters = KModes(n_clusters=4, max_iter=1000, dissim=False).fit(soybean)
soybean['Prediction'] = soyClusters.membship_

In [401]:
soyClusters.cost_

33.192613960358905